### Importação de bibliotecas

In [7]:
# Importando a biblioteca pandas, importante para tratamento e visualização dos dados importados
import pandas as pd
# Importando a biblioteca bigquery, para que possamos transferir os arquibvos tratados para um banco de dados.
from google.cloud import bigquery
# Importando a biblioteca zipfile para descompactar os arquivos importados via API do Kaggle.
import zipfile

In [43]:
# Download do conjunto de dados via API do Kaggle.
!kaggle datasets download -d olistbr/brazilian-ecommerce

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
brazilian-ecommerce.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
#Descompactando arquivo recebido
zip_file = 'brazilian-ecommerce.zip'
# Selecionando uma pasta chamada /csv/ para melhorar a organização.
destination_folder = 'csv/'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

In [39]:
df_geolocation = pd.read_csv('csv/olist_geolocation_dataset.csv')
df_customers = pd.read_csv('csv/olist_customers_dataset.csv')
df_items = pd.read_csv('csv/olist_order_items_dataset.csv')
df_payments = pd.read_csv('csv/olist_order_payments_dataset.csv')
df_reviews = pd.read_csv('csv/olist_order_reviews_dataset.csv')
df_orders = pd.read_csv('csv/olist_orders_dataset.csv')
df_products = pd.read_csv('csv/olist_products_dataset.csv')
df_sellers = pd.read_csv('csv/olist_sellers_dataset.csv')
df_name = pd.read_csv('csv/product_category_name_translation.csv')

### Carregamentos dos dados tratados para um Banco de Dados

In [6]:
df.to_gbq(destination_table='koru-dados.projeto2.your_table', project_id='koru-dados', if_exists='replace')

NameError: name 'df' is not defined